In [1]:
import pandas as pd
from datetime import datetime, timedelta
from datetime import date
from pandas.tseries.offsets import DateOffset
import calendar
from pandas.tseries.holiday import AbstractHolidayCalendar, Holiday, EasterMonday, Easter, weekend_to_monday
from pandas.tseries.offsets import Day, CustomBusinessDay

class RussianBusinessCalendar(AbstractHolidayCalendar):

    rules = [
        Holiday('New Year',  month=1, day=1),
        Holiday('New Year',  month=1, day=2),
        Holiday('Orthodox Christmas',  month=1, day=7, observance=weekend_to_monday),
        Holiday('Army day',  month=2, day=23, observance=weekend_to_monday),
        Holiday('Women day',  month=3, day=8, observance=weekend_to_monday),
        Holiday('Labour day',  month=5, day=1, observance=weekend_to_monday),
        Holiday('Victory day',  month=5, day=9, observance=weekend_to_monday),
        Holiday('Independence day',  month=6, day=12, observance=weekend_to_monday),
        Holiday('Day of unity',  month=11, day=4, observance=weekend_to_monday),
        Holiday('Pre New Year',  month=12, day=31),
    ]

    
 
def last_workday(year, month, bcs):
    start = datetime(2019, 1, 1)
    end =  datetime(2020, 1, 1)
    holidays = bcs.holidays(start, end)
    dt = date(year, month, calendar.monthrange(year, month)[1])
    dt -= timedelta(days=1)
    while dt.weekday() > 4 or dt in holidays:
        # Mon-Fri are 0-4
        dt -= timedelta(days=1)
    return dt

def pre_holiday(dt, bcs):
    start = datetime(2019, 1, 1)
    end =  datetime(2020, 1, 1)
    holidays = bcs.holidays(start, end)
    dt += timedelta(days=1)
    while dt.weekday() > 4 and dt not in holidays:
        dt += timedelta(days=1)
    if dt in holidays:
        return True
    return False

def day_type(dt, bcs):
    start = datetime(2019, 1, 1)
    end =  datetime(2020, 1, 1)
    holidays = bcs.holidays(start, end)
    dt += timedelta(days=1)
    if dt in holidays:
        return 'holiday'
    while dt.weekday() > 4 and dt not in holidays:
        dt += timedelta(days=1)
    if dt in holidays:
        return 'pre_holiday'
    return 'day'

def lag_workday(lag, dt, bcs):
    start = datetime(2019, 1, 1)
    end =  datetime(2020, 1, 1)
    holidays = bcs.holidays(start, end)
    for i in range(lag):
        dt -= timedelta(days=1)
        while dt.weekday() > 4 or dt in holidays:
            dt -= timedelta(days=1)
    return dt      
    
data = pd.read_csv("data.csv")
df = pd.DataFrame(data)

      
        
bcs = RussianBusinessCalendar()    
bcs.rules.append(Holiday('Labour Day', month=5, day=1))
print(df['Год/Месяц'].head())    
print(df['Январь'].head())    


start = datetime(2019, 1, 1)
end =  datetime(2020, 1, 1)
print(date.today().weekday())
holidays = bcs.holidays(start, end)
print(holidays)

dfDates = pd.DataFrame({"Days":pd.date_range(date(2020, 1, 1), periods=366).tolist()})
dfDates['Monday'] = dfDates['Days'].apply(lambda x: 1 if x.weekday() == 0 else 0 )
dfDates['Tuesday'] = dfDates['Days'].apply(lambda x: 1 if x.weekday() == 1 else 0 )
dfDates['Wednesday'] = dfDates['Days'].apply(lambda x: 1 if x.weekday() == 2 else 0 )
dfDates['Thursday'] = dfDates['Days'].apply(lambda x: 1 if x.weekday() == 3 else 0 )
dfDates['Friday'] = dfDates['Days'].apply(lambda x: 1 if x.weekday() == 4 else 0 )
dfDates['Saturday'] = dfDates['Days'].apply(lambda x: 1 if x.weekday() == 5 else 0 )
dfDates['Sunday'] = dfDates['Days'].apply(lambda x: 1 if x.weekday() == 6 else 0 )
dfDates['PreHoliday'] = dfDates['Days'].apply(lambda x: 1 if x.weekday() == 6 else 0 )
dfDates['Holiday'] = dfDates['Days'].apply(lambda x: 1 if pre_holiday(x, bcs) else 0 )
dfDates['Weekend'] = dfDates['Days'].apply(lambda x: 1 if x.weekday() in [5, 6] else 0 )
#calendar.monthrange(year, month)[1]
dfDates['LastMonthDay'] = dfDates['Days'].apply(lambda x: 1 if calendar.monthrange(x.year, x.month)[1] == x.day else 0 )
dfDates['LastBusinessDay'] = dfDates['Days'].apply(lambda x: 1 if last_workday(x.year, x.month, bcs) == x else 0 )
display(dfDates)


0    1999
1    2000
2    2001
3    2002
4    2003
Name: Год/Месяц, dtype: int64
0    1,2,3,4,6*,7,9,10,16,17,23,24,30,31
1     1,2,3,4,6*,7,8,9,15,16,22,23,29,30
2            1,2,6,7,8,13,14,20,21,27,28
3            1,2,5,6,7,12,13,19,20,26,27
4         1,2,3,5*,6,7,11,12,18,19,25,26
Name: Январь, dtype: object
3
DatetimeIndex(['2019-01-01', '2019-01-02', '2019-01-07', '2019-02-25',
               '2019-03-08', '2019-05-01', '2019-05-01', '2019-05-09',
               '2019-06-12', '2019-11-04', '2019-12-31', '2020-01-01'],
              dtype='datetime64[ns]', freq=None)


,Days,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,PreHoliday,Holiday,Weekend,LastMonthDay,LastBusinessDay
0,2020-01-01,0,0,1,0,0,0,0,0,0,0,0,0
1,2020-01-02,0,0,0,1,0,0,0,0,0,0,0,0
2,2020-01-03,0,0,0,0,1,0,0,0,0,0,0,0
3,2020-01-04,0,0,0,0,0,1,0,0,0,1,0,0
4,2020-01-05,0,0,0,0,0,0,1,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,2020-12-27,0,0,0,0,0,0,1,1,0,1,0,0
362,2020-12-28,1,0,0,0,0,0,0,0,0,0,0,0
363,2020-12-29,0,1,0,0,0,0,0,0,0,0,0,0
364,2020-12-30,0,0,1,0,0,0,0,0,0,0,0,1


In [2]:
#SBER loading a csv archive from finam.ru
data2 = pd.read_csv("sber_stocks.csv")
df2 = pd.DataFrame(data2)
df2 = df2[['CLOSE', 'TRADEDATE']]
print(df2.head())

   CLOSE   TRADEDATE
0  98.79  2013-03-25
1  97.20  2013-03-26
2  96.75  2013-03-27
3  98.59  2013-03-28
4  98.76  2013-03-29


In [4]:
df2['TRADEDATE'] = pd.to_datetime(df2['TRADEDATE'], format='%Y-%m-%d') 
df2 = df2[(df2['TRADEDATE']>start) & (df2['TRADEDATE']<end)]
df2['lag5'] = df2['TRADEDATE'].apply(lambda x: lag_workday(5, x, bcs))
df2['lag3'] = df2['TRADEDATE'].apply(lambda x: lag_workday(3, x, bcs))
df2['lag1'] = df2['TRADEDATE'].apply(lambda x: lag_workday(1, x, bcs))
df2['day_type'] = df2['TRADEDATE'].apply(lambda x: day_type(x, bcs))

week_df_mean = df2.groupby(df2['TRADEDATE'].dt.weekday).rolling(5, min_periods=1)['CLOSE'].mean()
#example of Monday group (weekday==0)
df3 = pd.DataFrame({'price':week_df_mean[0]})
df3 = df3.merge(df2, left_index=True, right_index=True)
print(df3)
week_df_max = df2.groupby(df2['TRADEDATE'].dt.weekday).rolling(5, min_periods=1).max()
week_df_min = df2.groupby(df2['TRADEDATE'].dt.weekday).rolling(5, min_periods=1).min()
week_df_median = df2.groupby(df2['TRADEDATE'].dt.weekday).rolling(5, min_periods=1).median()
week_df_std = df2.groupby(df2['TRADEDATE'].dt.weekday).rolling(5, min_periods=1).std()

week_df_mean = df2.groupby(df2['TRADEDATE'].dt.weekday).rolling(3, min_periods=1).mean()
week_df_max = df2.groupby(df2['TRADEDATE'].dt.weekday).rolling(3, min_periods=1).max()
week_df_min = df2.groupby(df2['TRADEDATE'].dt.weekday).rolling(3, min_periods=1).min()
week_df_median = df2.groupby(df2['TRADEDATE'].dt.weekday).rolling(3, min_periods=1).median()
week_df_std = df2.groupby(df2['TRADEDATE'].dt.weekday).rolling(3, min_periods=1).std()

week_df_mean = df2.groupby(df2['TRADEDATE'].dt.weekday).rolling(1, min_periods=1).mean()
week_df_max = df2.groupby(df2['TRADEDATE'].dt.weekday).rolling(1, min_periods=1).max()
week_df_min = df2.groupby(df2['TRADEDATE'].dt.weekday).rolling(1, min_periods=1).min()
week_df_median = df2.groupby(df2['TRADEDATE'].dt.weekday).rolling(1, min_periods=1).median()
week_df_std = df2.groupby(df2['TRADEDATE'].dt.weekday).rolling(1, min_periods=1).std()

week_df_mean_e = df2.groupby(df2['day_type']).expanding(5).mean()
week_df_max_e = df2.groupby(df2['day_type']).expanding(5).max()
week_df_min_e = df2.groupby(df2['day_type']).expanding(5).min()
week_df_median_e = df2.groupby(df2['day_type']).expanding(5).median()
week_df_std_e = df2.groupby(df2['day_type']).expanding(5).std()


week_df_mean_e = df2.groupby(df2['day_type']).expanding(3).mean()
week_df_max_e = df2.groupby(df2['day_type']).expanding(3).max()
week_df_min_e = df2.groupby(df2['day_type']).expanding(3).min()
week_df_median_e = df2.groupby(df2['day_type']).expanding(3).median()
week_df_std_e = df2.groupby(df2['day_type']).expanding(3).std()


week_df_mean_e = df2.groupby(df2['day_type']).expanding(1).mean()
week_df_max_e = df2.groupby(df2['day_type']).expanding(1).max()
week_df_min_e = df2.groupby(df2['day_type']).expanding(1).min()
week_df_median_e = df2.groupby(df2['day_type']).expanding(1).median()
week_df_std_e = df2.groupby(df2['day_type']).expanding(1).std()


           price   CLOSE  TRADEDATE       lag5       lag3       lag1 day_type
1461  196.800000  196.80 2019-01-14 2019-01-04 2019-01-09 2019-01-11      day
1466  201.985000  207.17 2019-01-21 2019-01-14 2019-01-16 2019-01-18      day
1471  204.556667  209.70 2019-01-28 2019-01-21 2019-01-23 2019-01-25      day
1476  207.292500  215.50 2019-02-04 2019-01-28 2019-01-30 2019-02-01      day
1481  208.634000  214.00 2019-02-11 2019-02-04 2019-02-06 2019-02-08      day
1486  210.112000  204.19 2019-02-18 2019-02-11 2019-02-13 2019-02-15      day
1491  209.850000  205.86 2019-02-25 2019-02-18 2019-02-20 2019-02-22      day
1496  209.010000  205.50 2019-03-04 2019-02-22 2019-02-27 2019-03-01      day
1500  206.960000  205.25 2019-03-11 2019-03-01 2019-03-05 2019-03-07      day
1505  205.340000  205.90 2019-03-18 2019-03-11 2019-03-13 2019-03-15      day
1510  207.302000  214.00 2019-03-25 2019-03-18 2019-03-20 2019-03-22      day
1515  209.670000  217.70 2019-04-01 2019-03-25 2019-03-27 2019-0